# 第4章: 言語解析

問題30から問題35までは、以下の文章`text`（太宰治の『走れメロス』の冒頭部分）に対して、言語解析を実施せよ。問題36から問題39までは、国家を説明した文書群（日本語版ウィキペディア記事から抽出したテキスト群）をコーパスとして、言語解析を実施せよ。

In [1]:
!pip install mecab-python3
!pip install ipadic

In [2]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

## 30. 動詞
文章`text`に含まれる動詞をすべて表示せよ。

In [3]:
import MeCab
import ipadic
import pandas as pd
import io
tagger = MeCab.Tagger(ipadic.MECAB_ARGS)

taged_text = tagger.parse(text)
f = io.StringIO(taged_text)

df = pd.read_csv(f, sep='\s', header=None)
df[['品詞', '品詞細分類1', '品詞細分類2', '品詞細分類3', '活用型', '活用形', '原形', '読み', '発音']] = df[1].str.split(',', expand=True)
df


<ipython-input-3-010fc147570e>:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(f, sep='\s', header=None)


,0,1,品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音
0,メロス,"名詞,一般,*,*,*,*,*",名詞,一般,*,*,*,*,*,None,None
1,は,"助詞,係助詞,*,*,*,*,は,ハ,ワ",助詞,係助詞,*,*,*,*,は,ハ,ワ
2,激怒,"名詞,サ変接続,*,*,*,*,激怒,ゲキド,ゲキド",名詞,サ変接続,*,*,*,*,激怒,ゲキド,ゲキド
3,し,"動詞,自立,*,*,サ変・スル,連用形,する,シ,シ",動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
4,た,"助動詞,*,*,*,特殊・タ,基本形,た,タ,タ",助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
...,...,...,...,...,...,...,...,...,...,...,...
62,で,"助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ",助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
63,あっ,"助動詞,*,*,*,五段・ラ行アル,連用タ接続,ある,アッ,アッ",助動詞,*,*,*,五段・ラ行アル,連用タ接続,ある,アッ,アッ
64,た,"助動詞,*,*,*,特殊・タ,基本形,た,タ,タ",助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
65,。,"記号,句点,*,*,*,*,。,。,。",記号,句点,*,*,*,*,。,。,。


In [4]:
verbs_df = df[df['品詞'] == '動詞']
verbs_df[0]

,0
3,し
14,除か
17,なら
21,し
29,わから
43,吹き
47,遊ん
49,暮し
51,来


In [5]:
#いやがらせ

tex ="すもももももももものうち"
taged_tex = tagger.parse(tex)
f = io.StringIO(taged_tex)

dfra = pd.read_csv(f, sep='\s', header=None)
dfra

<ipython-input-5-fba01d208dc8>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfra = pd.read_csv(f, sep='\s', header=None)


,0,1
0,すもも,"名詞,一般,*,*,*,*,すもも,スモモ,スモモ"
1,も,"助詞,係助詞,*,*,*,*,も,モ,モ"
2,もも,"名詞,一般,*,*,*,*,もも,モモ,モモ"
3,も,"助詞,係助詞,*,*,*,*,も,モ,モ"
4,もも,"名詞,一般,*,*,*,*,もも,モモ,モモ"
5,の,"助詞,連体化,*,*,*,*,の,ノ,ノ"
6,うち,"名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ"
7,EOS,None


## 31. 動詞の原型
文章`text`に含まれる動詞と、その原型をすべて表示せよ。

In [6]:
verbs_df = df[df['品詞'] == '動詞']
verbs_df[[0,"原形"]]

,0,原形
3,し,する
14,除か,除く
17,なら,なる
21,し,する
29,わから,わかる
43,吹き,吹く
47,遊ん,遊ぶ
49,暮し,暮す
51,来,来る


## 32. 「AのB」
文章`text`において、2つの名詞が「の」で連結されている名詞句をすべて抽出せよ。

In [7]:
len(df)

67

In [8]:
noun_phrages = []

for i in range(len(df) - 2):
  if df.iloc[i]['品詞'] == '名詞' and  df.iloc[i + 1][0] == 'の' and df.iloc[i + 2]['品詞'] == '名詞':
    noun_phrages.append(df.iloc[i][0] + 'の' + df.iloc[i + 2][0])

noun_phrages



['暴虐の王', '村の牧人']

## 33. 係り受け解析

文章`text`に係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

In [9]:
!pip install -U ginza

In [10]:
!pip install spacy==3.6

In [11]:
!python -m spacy download ja_ginza_electra

2025-04-30 07:39:45.692818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745998785.735373    9825 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745998785.748123    9825 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 07:39:45.813152: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 14

In [ ]:
!pip install -U numpy==1.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 74.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
import spacy
nlp = spacy.load('ja_ginza_electra')
doc = nlp(text)
doc

AttributeError: module 'numpy' has no attribute 'dtypes'

## 34. 主述の関係
文章`text`において、「メロス」が主語であるときの述語を抽出せよ。

## 35. 係り受け木
「メロスは激怒した。」の係り受け木を可視化せよ。

## 36. 単語の出現頻度

問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](/data/jawiki-country.json.gz)をコーパスと見なし、統計的な分析を行う。

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が"title"キーに、記事本文が"text"キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 37. 名詞の出現頻度
コーパスにおける名詞の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 38. TF・IDF
日本に関する記事における名詞のTF・IDFスコアを求め、TF・IDFスコア上位20語とそのTF, IDF, TF・IDFを表示せよ。

## 39. Zipfの法則
コーパスにおける単語の出現頻度順位を横軸、その出現頻度を縦軸として、両対数グラフをプロットせよ。